In [ ]:
import zipfile
import tarfile
import torch
from tqdm import tqdm
from torchaudio.io import StreamReader
import torchaudio
import time
from IPython.display import Audio, display

In [ ]:
def iterate_chunk(
    file_name,
    file_stream,
    chunk_size,
    target_sample_rate
):
    streamer = StreamReader(file_stream)    
    metadata = streamer.get_src_stream_info(0)
    
    streamer.add_basic_audio_stream(
        frames_per_chunk=int(chunk_size*target_sample_rate),
        sample_rate=target_sample_rate,
        decoder_option={
            "threads": "0", # Let ffmpeg decide
        }
    )

    for (chunk,) in streamer.stream():
        yield file_name, chunk.reshape(1, -1)

def iterate_tar(x):
    with tarfile.open(x, 'r') as tar:
        for member in tar.getmembers():
            start_time = time.time()
            if not member.isfile():
                continue

            file_content = tar.extractfile(member)
            file_name = member.name
            
            if file_content is None:
                logger.error(f"Error extracting file {file_name} from {x}")
                continue

            yield from iterate_chunk(
                file_name,
                file_content,
                target_sample_rate=16_000,
                chunk_size=30
            )


def iterate_zip(x):
    with zipfile.ZipFile(x, 'r') as zip_file:
        for file_info in zip_file.infolist():
            file_content = zip_file.open(file_info.filename)
            file_name = file_info.filename

            if file_info.is_dir():
                print(f'Found directory: {file_info}')

            print(f'found  {file_name}')

In [ ]:
zipf = '/home/romit/Downloads/Malayalam.zip'

In [ ]:
temp = iterate_zip(zipf)

In [ ]:
temp.

In [ ]:
tar_file = '/home/romit/Downloads/audio/P0001.tgz'

In [ ]:
idx = 0
total_data = []

for fn, data in tqdm(iterate_tar(tar_file)):
    # print(fn, data.shape)
    # total_data.append(data)
    idx += 1

    if idx == 1000:
        break

In [ ]:
display(Audio(total_data[0], rate=16000))